In [11]:
# Libraries to import

# The model is to select some articles to max the cost given a volume capacity 
# and a set of not essential items

import sys
import gurobipy as gp
import itertools
from gurobipy import GRB
from random import randint, seed

articles = ['pencil','chair','usb','notebook','book','waterbottle',
            'charger','toy','mouse','cellphone','laptop','pen',
            'ruler', 'eraser','paperclip','id']

no_essential = ['paperclip','chair','toy','ruler','mouse','waterbottle']

no_essential_index = []

for item in no_essential:
    no_essential_index.append(articles.index(item))


volumes = [8,36,48,4,16,7,31,48,28,30,41,24,50,13,6,31]

costs = [8, 36, 48, 4, 16, 7, 31, 48, 28, 30, 41, 24, 50, 13, 6, 31]


capacity = sum(volumes)



In [2]:
k = gp.Model()

items = [i for i in range(len(articles))]

s= 2

y = k.addVars(range(len(articles)),vtype=GRB.BINARY, name='yi')

k.setObjective(sum(y[i]*costs[i] for i in items), GRB.MAXIMIZE)

k.addConstr(gp.quicksum((y[i]*costs[i] for i in items)) <= capacity, name='Capacity')

k.addConstrs((y[i] == 1 for i in items if i not in no_essential_index), name='Essentials') 

k.addConstr(gp.quicksum(y[i] for i in items) <= 16-s, name='Knapsack_set')

Using license file C:\Users\alexa\gurobi.lic
Academic license - for non-commercial use only - expires 2021-03-06


<gurobi.Constr *Awaiting Model Update*>

In [3]:
k.optimize()


if k.status == GRB.OPTIMAL:
    #k.write('knapsack_example_Op.lp')
    obj = k.getObjective()
    print('P value','->',obj.getValue())
    var = k.getAttr('X',k.getVars())
    print('Articulos taken:')
    for i in range(len(articles)):
        if var[i] > 0:
            print(articles[i],'->','y->',var[i])
    print('Articles not taken:')
    for i in range(len(articles)):
        if var[i] == 0:
            print(articles[i],'->','y->',var[i])

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12 rows, 16 columns and 42 nonzeros
Model fingerprint: 0xa152ccbb
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 365.0000000
Presolve removed 12 rows and 16 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 408 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.080000000000e+02, best bound 4.080000000000e+02, gap 0.0000%
P value -> 408.0
Articulos taken:
pencil -> y-> 1.0
chair -> y-> 1.0
usb -> y-> 1.0
notebook -> y-> 1.0
book -> y-> 1.0
charger -> y-> 1.0
toy -> y-> 1.0
mouse -> y-> 1.

In [10]:
seed(124)
c = 1000
w = [randint(5,10) * 21 for i in range(100)]
p = [randint(100,120) * w[i] for i in range(100)]


n = gp.Model()

x = n.addVars(len(p),vtype=GRB.BINARY,name='x')

n.setObjective(sum(x[i]*p[i] for i in range(len(p))), GRB.MAXIMIZE)

n.addConstr(gp.quicksum((x[i]*w[i] for i in range(len(p)))) <= c, name='Capacity')

n.addConstr(sum(x[i] for i in range(len(p))) <= 90)

n.optimize()

sol = n.getObjective()

for i in range(len(p)):
    if x[i].X <.5:
        print(i,'--->',x[i].X)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 100 columns and 200 nonzeros
Model fingerprint: 0x31c3ac02
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+04, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 1e+03]
Found heuristic solution: objective 110481.00000
Presolve removed 2 rows and 100 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 118083 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.180830000000e+05, best bound 1.180830000000e+05, gap 0.0000%
0 ---> 0.0
1 ---> 0.0
2 ---> 0.0
3 ---> 0.0
4 ---> 0.0
5 ---> 0.0
6 ---> 0.0
7 ---> 0.0
8 ---> 0.0
9 ---> 0.0
10 ---> 0.0
12 ---> 0.0
13 ---> 0.0
14 ---> 0.0
15 -

In [14]:
try:
    # Sample data
    Groundset = range(10)
    objCoef = [32, 32, 15, 15, 6, 6, 1, 1, 1, 1]
    knapsackCoef = [16, 16,  8,  8, 4, 4, 2, 2, 1, 1]
    Budget = 33

    # Create initial model
    model = gp.Model("poolsearch")

    # Create dicts for tupledict.prod() function
    objCoefDict = dict(zip(Groundset, objCoef))
    knapsackCoefDict = dict(zip(Groundset, knapsackCoef))

    # Initialize decision variables for ground set:
    # x[e] == 1 if element e is chosen
    Elem = model.addVars(Groundset, vtype=GRB.BINARY, name='El')

    # Set objective function
    model.ModelSense = GRB.MAXIMIZE
    model.setObjective(Elem.prod(objCoefDict))

    # Constraint: limit total number of elements to be picked to be at most
    # Budget
    model.addConstr(Elem.prod(knapsackCoefDict) <= Budget, name='Budget')

    # Limit how many solutions to collect
    model.setParam(GRB.Param.PoolSolutions, 1024)
    # Limit the search space by setting a gap for the worst possible solution
    # that will be accepted
    model.setParam(GRB.Param.PoolGap, 0.10)
    # do a systematic search for the k-best solutions
    model.setParam(GRB.Param.PoolSearchMode, 2)

    # save problem
    model.write('poolsearch.lp')

    # Optimize
    model.optimize()

    model.setParam(GRB.Param.OutputFlag, 0)

    # Status checking
    status = model.Status
    if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
        print('The model cannot be solved because it is infeasible or '
              'unbounded')
        sys.exit(1)

    if status != GRB.OPTIMAL:
        print('Optimization was stopped with status ' + str(status))
        sys.exit(1)

    # Print best selected set
    print('Selected elements in best solution:')
    print('\t', end='')
    for e in Groundset:
        if Elem[e].X > .9:
            print(' El%d' % e, end='')
    print('')

    # Print number of solutions stored
    nSolutions = model.SolCount
    print('Number of solutions found: ' + str(nSolutions))

    # Print objective values of solutions
    print('******')
    for e in range(nSolutions):
        model.setParam(GRB.Param.SolutionNumber, e)
        print('%g ' % model.PoolObjVal, end='')
        if e % 15 == 14:
            print('')
    print('')

    # print fourth best set if available
    print('@@@@')
    if (nSolutions >= 4):
        model.setParam(GRB.Param.SolutionNumber, 3)

        print('Selected elements in fourth best solution:')
        print('\t', end='')
        for e in Groundset:
            if Elem[e].Xn > .9:
                print(' El%d' % e, end='')
        print('')

except gp.GurobiError as e:
    print('Gurobi error ' + str(e.errno) + ": " + str(e.message))

except AttributeError as e:
    print('Encountered an attribute error: ' + str(e))

Changed value of parameter PoolSolutions to 1024
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolGap to 0.1
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x435cf4bf
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective 65.0000000
Presolve time: 0.00s
Presolved: 1 rows, 10 columns, 10 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 6.587500e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds     